In [1]:
import pandas as pd
import numpy as np

# Import Participants
df = pd.read_csv("typeform/responses.csv",
                 sep=";", 
                 index_col=False,
                 names=['name', 'status', 'email','phone','faculty','location','consent','duplicate'],
                 converters={'phone': str})

# Drop columns
df = df.drop(['consent', 'duplicate'], axis=1)

# Drop single entries
#df = df[~df.email.str.contains("mail@student.maastrichtuniversity.nl")]

# Select Group
students = df.loc[df['status'] == "Student"]
staff = df.loc[df['status'] == "Staff"]
phds = df.loc[df['status'] == "PhD Student"]

In [3]:
# Randomize function
np.random.seed(5)

In [4]:
# Load all past matches to check for overlaps later

mar01 = pd.read_csv("history/walks_mar01.csv", usecols=["walk_id"])
mar07 = pd.read_csv("history/walks_mar07.csv", usecols=["walk_id"])
mar14 = pd.read_csv("history/walks_mar14.csv", usecols=["walk_id"])

# Merge all in history
history =  pd.concat([mar01, mar07, mar14], ignore_index=False, axis=0)

In [5]:
def createtable(name):
    # Randomize list
    df = name.sample(frac=1).reset_index()
    # Create pairs
    new_index = df.index.to_numpy().reshape(df.shape[0]//2, 2)[:, ::-1].reshape(-1,)
    out = pd.concat([df,df.reindex(new_index).add_suffix('_peer').reset_index(drop=True)], axis=1)
    out = out.drop(['index', 'index_peer'], axis=1)
    # Unique ID
    out['walk_id'] = out[['email','email_peer']].sum(axis=1).map(hash).abs()
    return out

In [7]:
students = createtable(students)
phds = createtable(phds)
staff = createtable(staff)

In [13]:
# Concatenate dataframes
walks = pd.concat([students, staff, phds], ignore_index=False, axis=0)

In [15]:
# Find duplicates
set(walks['walk_id']).intersection(set(history['walk_id']))

set()

In [16]:
len(walks)

2

In [9]:
walks.to_csv('walks.csv', index=False)